# Задание 1

In [91]:
import os

import os
import json
import pandas as pd
from bs4 import BeautifulSoup

In [92]:
# Загрузка файла
def upload_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    return soup

In [93]:
def create_dataframe(data):
    try:
        return pd.DataFrame(data)
    except:
        print('Error while creating dataframe')
        return None

In [ ]:
def save_json(data, path):
    with open(path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

In [95]:
# загрузка информации из файла в словарь
def get_html_data(filepath):
    soup = upload_file(filepath)

    city = soup.find('span').text.strip().split(': ')[1]
    title = soup.find('h1', class_='title').text.strip().split(':')[1]
    address = soup.find('p', class_='address-p').text.strip().replace('Улица:', '').replace('Индекс:', '').strip()
    floors = int(soup.find('span', class_='floors').text.strip().split(': ')[1])
    year_built = int(soup.find('span', class_='year').text.strip().split(' ')[-1])
    parking = soup.find_all('span')[3].text.strip().split(':')[1]
    rating = float(soup.find_all('span')[4].text.strip().split(':')[1])
    views = int(soup.find_all('span')[5].text.strip().split(': ')[1])
    
    result = {
        'city': city,
        'title': title,
        'address': address,
        'floors': floors,
        'year_built': year_built,
        'parking': parking,
        'rating': rating,
        'views': views
    }

    return result

In [96]:
# функция получения данных из всех файлов
def read_files(folderpath):
    data_list = list()

    for filename in os.listdir(folderpath):
        if filename.endswith('.html'):
            file_path = os.path.join(folderpath, filename)
            data = get_html_data(file_path)
            data_list.append(data)

    return data_list

In [97]:
def sort_data(df):
    return df.sort_values(by='rating')

In [98]:
def filtre_data(df):
    return df[df['parking'] == 'нет']

In [99]:
def calculate_statistic(df):
    _sum = df['views'].sum()
    _min = df['views'].min()
    _max = df['views'].max()
    _avg = df['views'].mean()

    return _sum, _min, _max, _avg

In [100]:
def calculate_frequency(df):
    return df['city'].value_counts()

In [ ]:
def print_result(title, data):
      if type(data) == list:
            print('\n','='*15,'\n',
                  title,'\n\n',
                  *data,'\n',
                  '='*15,'\n')
      else:
            print('\n','='*15,'\n',
                  title,'\n\n',
                  data,'\n',
                  '='*15,'\n')

In [117]:
def first_task(folderpath):
    # получим список словарей с данными из html
        data = read_files(folderpath)

        df = create_dataframe(data)
        # отсортируйте значения по одному из доступных полей
        sorted_df = sort_data(df)
        print_result('Sorted df:', sorted_df)

        # выполните фильтрацию по другому полю (запишите результат отдельно)
        filtre_df = filtre_data(df)
        print_result('Filtred df:', filtre_df)
        # для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
        summ, minn, maxx, avgg = calculate_statistic(df)
        print_result('Params:\n',[
                     f'sum: {summ}',
                     f'min: {minn}',
                     f'max: {maxx}',
                     f'avg: {avgg}'])

        # для одного текстового поля посчитайте частоту меток
        frequency = calculate_frequency(df)
        print_result('Frequency:', frequency)

        save_json(data,'./Answers/first_task/first_task.json')
        print('Saving data to json True')


In [118]:
if __name__ == '__main__':
    first_task('./55/1')


 Sorted df: 

                          city                title  \
6                      Бланес  \n     Кинотеатр 69   
40                       Рига       \n     Отель 7   
23          Алькала-де-Энарес     \n     Логово 92   
13                      Минск     \n     Тенета 55   
17                   Алма-Ата      \n     Башня 80   
30              Сан-Себастьян       Детский сад 47   
35                    Белград      \n     Вилла 26   
1                   Картахена              Ферма 6   
22                    Душанбе     \n     Пандус 52   
7                   Барселона           Лосятник 9   
0                      Овьедо  \n     Переправа 51   
3                      Мадрид              Петля 1   
10                   Аликанте          Кладбище 98   
39                    Кадакес             Ангар 72   
28                   Камбадос          Дуплянка 43   
24                     Загреб             Вилла 86   
38                  Подгорица   \n     Гусятник 58   
5           

# Задание 2

In [122]:
import os

import os
import json
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def save_json(data, path):
    with open(path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

In [121]:
# Загрузка файла
def upload_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    return soup

In [123]:
def create_dataframe(data):
    try:
        return pd.DataFrame(data)
    except:
        print('Error while creating dataframe')
        return None

In [ ]:
# загрузка информации из файла в словарь
def get_html_data(filepath):
    soup = upload_file(filepath)
    products = []

    for product_item in soup.find_all('div', class_='product-item'):
        product = {}
        product_name = product_item.find('span').text.strip()
        product['name'] = product_name
        
        price_text = product_item.find('price').text.strip().replace('₽', '').replace(' ', '')
        product['price'] = int(price_text) if price_text.isdigit() else None
        
        product['ram'] = None
        product['camera'] = None
        product['acc'] = None
        product['processor'] = None
        product['sim'] = None
        product['matrix'] = None
        product['resolution'] = None
        
        for el in product_item.find_all('li'):
            char_type = el.get('type')
            char_value = el.text.strip()
            if char_type in product:
                product[char_type] = char_value
            
        products.append(product)

    return products

In [128]:
# функция получения данных из всех файлов
def read_files(folderpath):
    data_list = list()

    for filename in os.listdir(folderpath):
        if filename.endswith('.html'):
            file_path = os.path.join(folderpath, filename)
            data = get_html_data(file_path)
            data_list.extend(data)

    return data_list

In [142]:
def second_task(folderpath):
    data = read_files(folderpath)
    df = create_dataframe(data)

    sorted_df = df.sort_values(by='price', ascending=True)
    print(sorted_df)

    filtered_df = df[df['sim'] == '3 SIM']
    print(filtered_df)

    price_sum = df['price'].sum()
    price_min = df['price'].min()
    price_max = df['price'].max()
    price_avg = df['price'].mean()

    print(f"Sum: {price_sum}, Min: {price_min}, Max: {price_max}, Average: {price_avg}")

    processor_frequency = df['processor'].value_counts()
    print(processor_frequency)

    save_json(data, './Answers/second_task/second_task.json')


In [143]:
second_task('./55/2')

                         name   price    ram camera          acc  processor  \
1359          5.9" HGST 224GB   10558   None  26 MP  3160 мА * ч  4x1.3 ГГц   
2315       5.8" SanDisk 224GB   10841   None  65 MP  3231 мА * ч  2x1.5 ГГц   
1413      5.0" Gigabyte 128GB   10932   6 GB  62 MP  4965 мА * ч  8x1.8 ГГц   
1744     7.3" Blackberry 16GB   11009   6 GB  64 MP  7262 мА * ч  6x1.4 ГГц   
1647   5.5" Google Pixel 96GB   11152  10 GB  72 MP         None  6x1.9 ГГц   
...                       ...     ...    ...    ...          ...        ...   
665           7.0" HGST 240GB  498793   None   None  5403 мА * ч  4x4.5 ГГц   
2200  5.5" Silicon Power 64GB  498871  14 GB   None         None  2x3.8 ГГц   
708        6.1" Gigabyte 32GB  498967   8 GB  72 MP         None       None   
2247           6.5" Dell 48GB  499346   None   None  6688 мА * ч  4x3.1 ГГц   
19          7.5" Micron 112GB  499709  12 GB  46 MP         None  2x2.3 ГГц   

        sim  matrix resolution  
1359  2 SIM    OLE